In [1]:
import tensorflow as tf
import matplotlib as plt
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))



C:\Users\Suhan Acharya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Suhan Acharya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Suhan Acharya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Suhan Acharya\anaconda3\lib\site-packages\tensorflow\python\framework\dty

In [2]:
DESIRED_ACCURACY = 0.95
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>DESIRED_ACCURACY):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [15]:
with tf.device("/device:GPU:0"):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(26, activation='softmax')
    ])

In [16]:
    model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 71, 71, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 35, 35, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 39200)            

In [17]:
    from tensorflow.keras.optimizers import RMSprop 

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [21]:
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   shear_range=0.2,
                                   zoom_range=0.5,
                                   fill_mode='nearest',)

    train_generator = train_datagen.flow_from_directory(
        "signs_data",  
        target_size=(300, 300), 
        batch_size=10,
        class_mode='sparse')

#validation_datagen = ImageGenerator(rescale=1/255)


#validation_generator = validation_datagen.flow_from_directory(
#    "sign_data/testing",
#    target_size = (150,150),
#    batch_size = 10,
#    class_mode='sparse')



Found 211 images belonging to 26 classes.


In [22]:
    history = model.fit_generator(train_generator, epochs=100, verbose=1, callbacks=[callbacks]) #, validation_data = validation_generator)

Epoch 1/100
22/22 [==============================] - 10s 454ms/step - loss: 2.1244 - acc: 0.3460
Epoch 2/100
22/22 [==============================] - 9s 412ms/step - loss: 2.0742 - acc: 0.4218
Epoch 3/100
22/22 [==============================] - 9s 402ms/step - loss: 1.8265 - acc: 0.4597
Epoch 4/100
22/22 [==============================] - 9s 416ms/step - loss: 1.9146 - acc: 0.4171
Epoch 5/100
22/22 [==============================] - 9s 412ms/step - loss: 1.6364 - acc: 0.5118
Epoch 6/100
22/22 [==============================] - 9s 408ms/step - loss: 1.5375 - acc: 0.5687
Epoch 7/100
22/22 [==============================] - 9s 405ms/step - loss: 1.5751 - acc: 0.5355
Epoch 8/100
22/22 [==============================] - 9s 407ms/step - loss: 1.3794 - acc: 0.5545
Epoch 9/100
22/22 [==============================] - 9s 403ms/step - loss: 1.4056 - acc: 0.5829
Epoch 10/100
22/22 [==============================] - 9s 406ms/step - loss: 1.3414 - acc: 0.5640
Epoch 11/100
22/22 [==================

KeyboardInterrupt: 

In [32]:
import numpy as np

img = tf.keras.preprocessing.image.load_img("test3.jpg", target_size=(300, 300))
  
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images)

print(classes)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0.]]


In [33]:
acc = history.history['accuracy']
#val_acc = model.history['val_accuracy']
loss = history.history['loss']
#val_loss = model.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
#plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.plot(epochs, loss, 'b', label='Training Loss')
#plt.title('Training and validation accuracy')
plt.title('Training accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

NameError: name 'history' is not defined